In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
import gym
import gym_tictactoe
from timeit import Timer
from sklearn.preprocessing import LabelBinarizer
from os import path

In [2]:
DEBUG = False

n_input = (3 * 3 * 3) * 3
n_hidden = 100
n_output = 3 * 3 * 3
learning_rate = 0.01

checkpoint_path = './my_dqn_tictactoe_h{}_lr{}.ckpt'.format(n_hidden, learning_rate)
initializer = tf.contrib.layers.variance_scaling_initializer()

env = gym.make('tictactoe-v0')
timer = Timer()

In [3]:
encoder = LabelBinarizer()
encoder.fit(np.array([[0], [1], [2]]))

def convert_game_to_x_state(obs):
    # gym_tictactoe now supports int-encoded world
    world = np.array(obs, dtype=np.float32)
    data = list(map(lambda x: [x], world.flatten()))
    return encoder.transform(data).flatten()

def convert_action_to_step(action, player):
    action = int(action)
    val = 0
    multiplier = 1
    while action:
        val += (action%3)*multiplier
        multiplier *= 10
        action //= 3
    
    return str(player) + str(val).zfill(3)

In [4]:
player_scopes = ['actor', 'critic']
all_logits = []
outputs = []
ys = []
all_network_trainable_vars_by_name = []

X_state = tf.placeholder(shape=(None, n_input), dtype=tf.float32)
global_step = tf.Variable(0, trainable=False, name='global_step')

for scope in player_scopes:
    with tf.variable_scope(scope) as tf_scope:
        hidden = fully_connected(X_state, n_hidden, activation_fn=tf.nn.relu, weights_initializer=initializer)
        logits = fully_connected(hidden, n_output, activation_fn=None, weights_initializer=initializer)
        all_logits.append(logits)

        output = tf.contrib.layers.softmax(logits)
        outputs.append(output)
        
        y = tf.to_float(tf.multinomial(tf.log(output), num_samples=n_output))
        ys.append(y)
        
        trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope)
        network_trainable_vars_by_name = { var.name[len(tf_scope.name):]: var for var in trainable_vars }
        all_network_trainable_vars_by_name.append(network_trainable_vars_by_name)

actor_net = outputs[0]
critic_net = outputs[1]

actor_vars = all_network_trainable_vars_by_name[0]
critic_vars = all_network_trainable_vars_by_name[1]

copy_ops = [actor_var.assign(critic_vars[var_name]) for var_name, actor_var in actor_vars.items()]
copy_critic_to_actor = tf.group(*copy_ops)

X_action = tf.placeholder(tf.int32, shape=[None])
q_value = tf.reduce_sum(critic_net * tf.one_hot(X_action, n_output), axis=1, keepdims=True)

y = tf.placeholder(tf.float32, shape=[None, 1])
cost = tf.reduce_mean(tf.square(y - q_value))

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(cost, global_step=global_step)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

file_writer = tf.summary.FileWriter('logs', tf.get_default_graph())
file_writer.close()

In [45]:
from collections import deque

eps_min = 0.05
eps_max = 1.0
eps_decay_steps = 100000
replay_mem_size = 1000000
replay_mem = deque([], maxlen=replay_mem_size)


def epsilon_greedy(q_values, step):
    epsilon = max(eps_min, eps_max - (eps_max-eps_min)*step/eps_decay_steps)
    if np.random.rand() < epsilon:
        return np.random.randint(n_output)
    else:
        return np.argmax(q_values)

def sample_mem(batch_size):
    indices = np.random.permutation(len(replay_mem))[:batch_size]
    cols = [[], [], [], [], []]
    for idx in indices:
        memory = replay_mem[idx]
        for col, value in zip(cols, memory):
            col.append(value)
    cols = [np.array(col) for col in cols]
    return (cols[0], cols[1], cols[2].reshape(-1, 1), cols[3], cols[4].reshape(-1, 1))

In [21]:
def play_with_human(env):
    obs = env.reset()
    done = False
    player = 0
    with tf.Session() as sess:
        saver.restore(sess, checkpoint_path)
        with tf.variable_scope('actor'):
            while not done:
                env.render()
                step = input()
                obs, reward, done, info = env.step('{}{}'.format(player%2+1, step))

                if done:
                    break

                player += 1
                state = convert_game_to_x_state(obs)
                action = np.argmax(actor_net.eval(feed_dict={X_state: [state]}))
                obs, reward, done, info = env.step(convert_action_to_step(action, player%2+1))

                player += 1
    env.render()

In [46]:
n_steps = 1000000
training_start = 1000
training_interval = 3
save_steps = 1000
copy_steps = 100
print_steps = 1000
discount_rate = 0.95
batch_size = 50
done = True

In [ ]:
iteration = 0
done = True

with tf.Session() as sess:
    if path.exists(checkpoint_path + '.meta'):
        saver.restore(sess, checkpoint_path)
    else:
        init.run()
    
    s_time = timer.timer()
    while True:
        step = global_step.eval()
        if step >= n_steps:
            break
        
        iteration += 1
        if done:
            obs = env.reset()
            state = convert_game_to_x_state(obs)
            player = 0
        
        q_values = actor_net.eval(feed_dict={X_state: [state]})
        action = epsilon_greedy(q_values, step)
        
        # Actor plays
        obs, reward, done, info = env.step(convert_action_to_step(action, player+1))
        next_state = convert_game_to_x_state(obs)
        player = (player+1) % 2
        
        # memorize
        replay_mem.append((state, action, reward, next_state, 1.0-done))
        state = next_state
        
        if iteration % print_steps == 0:
            e_time = timer.timer()
            print('Steps:{}'.format(step), 'Time:{0:.2f}s'.format(e_time-s_time))
            s_time = e_time
        
        if iteration % training_interval != 0:
            continue
        
        # Critic learns
        X_state_val, X_action_val, rewards, X_next_state, continues = (sample_mem(batch_size))
        next_q_values = actor_net.eval(feed_dict={X_state: X_next_state})
        max_next_q_values = np.max(next_q_values, axis=1, keepdims=True)
        y_val = rewards + continues * discount_rate * max_next_q_values
        training_op.run(feed_dict={X_state: X_state_val, X_action: X_action_val, y: y_val})
        
        if step % copy_steps:
            copy_critic_to_actor.run()
            
        if step % save_steps:
            saver.save(sess, checkpoint_path)

INFO:tensorflow:Restoring parameters from ./my_dqn_tictactoe_h100_lr0.01.ckpt


In [42]:
play_with_human(env)

INFO:tensorflow:Restoring parameters from ./my_dqn_tictactoe_h100_lr0.01.ckpt
- - -    - - -    - - -    
- - -    - - -    - - -    
- - -    - - -    - - -    
000
x - -    - - -    - - -    
- - -    - - -    - - -    
- - o    - - -    - - -    
011
x - -    - - -    - - o    
- x -    - - -    - - -    
- - o    - - -    - - -    
112
x - -    - - -    - - o    
- x -    - - -    - - -    
- - o    - x -    - - o    
012
x - -    - - -    - - o    
- x -    - - -    - - -    
- x o    - x -    - - o    


In [ ]:
with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    print(global_step.eval())

In [43]:
env.render()

x - -    - - -    - - o    
- x -    - - -    - - -    
- x o    - x -    - - o    


In [44]:
env._done

True